<a href="https://colab.research.google.com/github/nicoerdmann/Termofluidos/blob/main/Tarea_1_Nico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Primero comenzamos instalando las librerías.

In [ ]:
pip install Coolprop

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 3.0 MB/s eta 0:00:00


In [ ]:
pip install fluids

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 5.5 MB/s eta 0:00:00


In [ ]:
import CoolProp.CoolProp as cp
import fluids as fld
import numpy as np
from scipy import constants
from scipy.constants import g
from fluids.units import *
import pint

#Problema #2

In [ ]:
u = pint.UnitRegistry()

#Importante definir las unidades de medida
Diametro = (1.25 / 39.37) * u.m
vel_promedio = 100 / 3.281 * (u.m/u.s)
Largo = 300 * u.m
Area_transversal = (np.pi*Diametro**2)/4
V_dot = vel_promedio * Area_transversal

#Coolprop no permite trabajar con unidades, estos los dejamos como magnitudes escalares
Temperatura = 20 + 273.15
P_0 = 1 * 6895
Diametro, vel_promedio, Largo, Area_transversal, V_dot, Temperatura, P_0

(<Quantity(0.0317500635, 'meter')>,
 <Quantity(30.4785126, 'meter / second')>,
 <Quantity(300, 'meter')>,
 <Quantity(0.000791733603, 'meter ** 2')>,
 <Quantity(0.0241308626, 'meter ** 3 / second')>,
 293.15,
 6895)

a) Comenzamos calculando la pérdida de carga de la tubería

In [ ]:
# Definir el fluido como gas natural y la tuberia de cobre
fluido = 'Methane'
material = 'Copper'

#Obtenemos la rugosidad del material, la densidad (rho) y el mu
epsilon = fld.material_roughness(material)*u.m
rho = cp.PropsSI('D','T', Temperatura, 'P',P_0, fluido)*u.kg/u.m**3
mu = cp.PropsSI('V','T', Temperatura, 'P',P_0, fluido)*u.Pa*u.s


Determinamos el numero de Reynolds:

In [ ]:
Re = (fld.Reynolds(Diametro, rho, vel_promedio, mu)).to('dimensionless') #Reynolds es un numero adimensional
Re

<Quantity(3983.77444, 'dimensionless')>

Determinamos el factor de fricción:

In [ ]:
f = fld.friction.friction_factor(Re, eD=epsilon/Diametro, Method='Colebrook')
f

<Quantity(0.102648606, 'dimensionless')>

La pérdida de carga debido a la fricción puede calcularse a través de la siguiente fórmula,

$\Delta P_{\text{friction}} =f \frac{L}{D} \frac{\rho V^{2}}{2}$

In [ ]:
DELTA_P_f = ((f * Largo * rho * vel_promedio**2)/(2 * Diametro)).to(u.kPa)
DELTA_P_f

<Quantity(20.4470305, 'kilopascal')>

b) Primero, para entender el grado de error introducido por el supuesto, debemos ver cuáles serán nuestras pérdidas de cargas considerando un fluido compresible, es decir, que cambia su densidad.
Para eso, haremos divisiones a lo largo de la tubería e iremos evaluando las perdidas de carga utilizando las densidades en cada punto.

In [ ]:
P_0 = 1 * 6895 * u.Pa
#divisiones de la tuberia
numero_segmentos = 50
longitud_segmento = Largo / numero_segmentos

#perdida de presion acumulada
DP_total = 0

# Sumando los segmentos
for i in range(numero_segmentos):

    Re = (fld.Reynolds(Diametro.magnitude, rho.magnitude, vel_promedio.magnitude, mu.magnitude))
    f = fld.friction.friction_factor(Re, eD=(epsilon.magnitude/Diametro.magnitude), Method='Colebrook')

    # Calcular la pérdida de presión en el segmento actual
    DP_segmento = (f * longitud_segmento * rho * vel_promedio**2) / (2 * Diametro)
    DP_total += DP_segmento

    # Actualizar la presión y propiedades del fluido
    P_actual = P_0 - DP_total
    rho = cp.PropsSI('D', 'T', Temperatura, 'P', P_actual.magnitude, fluido) * u.kg/u.m**3
    mu = cp.PropsSI('V', 'T', Temperatura, 'P', P_actual.magnitude, fluido) * u.Pa*u.s

DP_total = DP_total.to(u.kPa)
print(f"Pérdida total de presión: {DP_total:.2f}")

Pérdida total de presión: 5.88 kilopascal


In [ ]:
#Considerando esto, el grado de error al asumir densidad cte es:
error = (DELTA_P_f - DP_total)/ DP_total * 100
error

<Quantity(247.891973, 'dimensionless')>